# Hopfield 🫠

https://towardsdatascience.com/hopfield-networks-neural-memory-machines-4c94be821073

To use an Hopfield Network for a Binary image classification task we stored in the network some labeled patterns and then predict the label of a new image with the label of the pattern to which it converges.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pathlib

from tqdm import tqdm
import random
import pygame

import tensorflow as tf
from keras import utils, layers

## Hopfield Class

Definition of the Hopfield network class

In [ ]:
class Hopfield_Net:  # network class
    # init ialize network variables and memory
    def __init__(self, input, state):

        # patterns for network training / retrieval
        self.memory = np.array(input)
        # single vs. multiple memories
        if self.memory.size > 1:
            self.n = self.memory.shape[1]
        else:
            self.n = len(self.memory)
        # network construction
        self.state = np.array(state)  # state vector
        self.weights = np.zeros((self.n, self.n))  # weights vector
        self.energies = []  # container for tracking of energy

        self.pointer = 0
        self.order = list(range(self.n))
        random.shuffle(self.order)

        self.is_over = False

    def network_learning(self):  # learn the pattern / patterns
        self.weights = (
            (1 / self.memory.shape[0]) * self.memory.T @ self.memory
        )  # hebbian learning
        np.fill_diagonal(self.weights, 0)

    def update_network_state(self, n_update):  # update network
        for _ in range(n_update):  # update n neurons randomly

            self.rand_index = self.order[self.pointer]

            # Compute activation for randomly indexed neuron
            self.index_activation = np.dot(self.weights[self.rand_index, :], self.state)
            # threshold function for binary state change
            if self.index_activation < 0:
                self.state[self.rand_index] = -1
            else:
                self.state[self.rand_index] = 1

        self.pointer = (self.pointer + 1) % self.n

    def compute_energy(self):  # compute energy
        self.energy = -0.5 * np.dot(np.dot(self.state.T, self.weights), self.state)
        self.energies.append(self.energy)


## HyperParam

In [ ]:
img_size = 64
train_size, test_size = 8, 24


## Dataset

In [ ]:
data_directory = pathlib.Path("downloads", "CatsDogs")
seed = 42

Downloading the dataset: as a big dataset is not needed for this network we focus just on a small sample.

In [ ]:
ds = utils.image_dataset_from_directory(
    data_directory,
    color_mode="grayscale",
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=1,
).take(train_size + test_size)


Converting pixels to bits

In [ ]:
normalization_layer = layers.Rescaling(scale=1.0 / 255, offset=-0.5001)
normalized_ds = ds.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
images = []
labels = np.array([])
for batch in tqdm(normalized_ds):
    # sobel=tf.image.sobel_edges(batch[0])
    # sobel_x=np.asarray(sobel[0,:,:,:,1])
    arr = np.squeeze(tf.math.sign(batch[0]))
    images.append(arr)
    labels = np.append(labels, (int(batch[1][0])))
images = np.array(images)
images = images.reshape((-1, img_size * img_size))


In [ ]:
plt.imshow(images[3].reshape((img_size, img_size)))
plt.show()


## Testing step by step

Setting in `memories_list` the images to be stored and in `test_image` the image to be tested

In [ ]:
memories_list=images[:4]
test_index=27
test_image=images[test_index]
memories_list[0].shape

Initializing the network

In [ ]:
H_Net = Hopfield_Net(memories_list, test_image)
H_Net.network_learning()


Plotting weights matrix

In [ ]:
plt.figure("weights", figsize=(10, 7))
plt.imshow(H_Net.weights, cmap="RdPu")  #
plt.xlabel("Each row/column represents a neuron, each square a connection")

plt.title(f" {img_size*img_size} Neurons - {img_size*img_size*img_size*img_size} unique connections", fontsize=15)
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])


Computing the stable state

In [ ]:
while not H_Net.is_over:
    old_state = np.array(H_Net.state)
    for _ in range(img_size):
        H_Net.update_network_state(img_size)
    H_Net.compute_energy()
    if np.all(H_Net.state == old_state):
        H_Net.is_over = True


In [ ]:
# plot energies
plt.figure("Energy", figsize=(6, 4))
x = np.arange(len(H_Net.energies))
plt.scatter(x, np.array(H_Net.energies), s=1, color="red")
plt.xlabel("Generation")
plt.ylabel("Energy")
plt.title("Network Energy over Successive Generations", fontsize=15)
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])


Tested image

In [ ]:
plt.imshow(test_image.reshape((img_size, img_size)))
plt.show()


Stable state reached

In [ ]:
plt.imshow(H_Net.state.reshape((img_size, img_size)))
plt.show()


> *Binary Hopfield networks (BHNs) are prone to ‘spurious’ minima. If memories learned by a BHN are too similar, or if too many pattern vectors are learned, the network risks converging to an in-between memory, some combination of learned patterns; in other words, the network will fail to discriminate between patterns and becomes useless.*

Checking if it is a spurius state or if the network predicted well

In [ ]:
spurius_state = True
for i, image in enumerate(memories_list):
    if np.all(H_Net.state == image):
        if labels[i] != labels[test_index]:
            print("error!")
        else:
            print("Predicted well!")
        spurius_state = False

if spurius_state == True:
    print("spurius state!")


## Exploratory experiments

As any pattern stored define a stable state, this network should have always zero training error.

In [ ]:
while True:
    training_and_test_indexis = list(range(train_size+test_size))
    random.shuffle(training_and_test_indexis)
    train_indexis = training_and_test_indexis[:train_size]
    test_indexis = training_and_test_indexis[train_size:]
    # check if there are both cats and dogs in train and test
    if all(v in labels[train_indexis] for v in [0, 1]) and all(
        v in labels[test_indexis] for v in [0, 1]
    ):
        break
print(train_indexis, test_indexis)


In [ ]:
memories_list=images[train_indexis] 
memories_labels=labels[train_indexis]
test_labels=labels[test_indexis]
print(memories_labels,test_labels)

### Train

In [ ]:
n_errors = 0
n_spurius_states = 0

for index in tqdm(train_indexis):

    # initialize
    H_Net = Hopfield_Net(memories_list, images[index])
    H_Net.network_learning()

    # convergence
    while not H_Net.is_over:
        old_state = np.array(H_Net.state)
        for _ in range(img_size):
            H_Net.update_network_state(img_size)
        if np.all(H_Net.state == old_state):
            H_Net.is_over = True

    # label check
    spurius_state = True
    for i, image in enumerate(memories_list):
        if np.all(H_Net.state == image):  # find the image to which had converged
            if memories_labels[i] != labels[index]:
                n_errors += 1
            spurius_state = False
    if spurius_state:
        n_spurius_states += 1

print("PERFORMANCE ON THE TRAINING SET:")
print(f"Number of errors: {n_errors}")
print(f"Number of spurius states reached: {n_spurius_states}")


### Test

In [ ]:
n_errors = 0
n_spurius_states = 0

for index in tqdm(test_indexis):

    # initialize
    H_Net = Hopfield_Net(memories_list, images[index])
    H_Net.network_learning()

    # convergence
    while not H_Net.is_over:
        old_state = np.array(H_Net.state)
        for _ in range(img_size):
            H_Net.update_network_state(img_size)
        if np.all(H_Net.state == old_state):
            H_Net.is_over = True

    # label check
    spurius_state = True
    for i, image in enumerate(memories_list):
        if np.all(H_Net.state == image):  # find the image to which had converged
            if memories_labels[i] != labels[index]:
                n_errors += 1
            spurius_state = False
    if spurius_state:
        n_spurius_states += 1

print("PERFORMANCE ON THE TEST SET:")
print(f"Number of errors: {n_errors}")
print(f"Number of spurius states reached: {n_spurius_states}")


In practice, even with a small number of patterns stored there is the risk that a some of the learned pattern converge instead to a spurius pattern.

It seems necessary to choose the patterns stored trying to minimizing the risk of spurius states. To do so the patterns stored need to be as dissimilar as possible.

So, we define an enreached algorithm using an Hopfield Network for binary image classification:

It take as inputs a training set of labeled images.
It has one parameters: **p** that is the number of patterns to be stored (we assume it to be even).

First it choose **p** images from the training set, ensuring that the proportion of labels is fair and maximizing the dissimilarity between the two less dissimilar examples.

Then it stored these patterns with an Hopfield Network.

### Hyperparameter tuning

In [ ]:
# for i in range(len(images)):
#     print(i)
#     plt.imshow(images[i].reshape((img_size, img_size)))
#     plt.show()


### Risk estimation

### Animation

In [ ]:
#parameters for pygame

# Draw it all out, updating board each update iteration
cellsize = 5

pygame.init()  # initialize pygame
# set dimensions of board and cellsize -  28 X 28  ~ special display surface
surface = pygame.display.set_mode((img_size * cellsize, img_size * cellsize))
pygame.display.set_caption("   ")

In [ ]:
# kill pygame if user exits window
Running = True
# main animation loop
try:
    while Running:
        for event in pygame.event.get():
            pygame.time.wait(1)
            if event.type == pygame.QUIT:
                Running = False

                # plot weights matrix
                plt.figure("weights", figsize=(10, 7))
                plt.imshow(H_Net.weights, cmap="RdPu")  #
                plt.xlabel("Each row/column represents a neuron, each square a connection")

                plt.title(" 4096 Neurons - 16,777,216 unique connections", fontsize=15)
                plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

                # plot energies
                plt.figure("Energy", figsize=(10, 7))
                x = np.arange(len(H_Net.energies))
                plt.scatter(x, np.array(H_Net.energies), s=1, color="red")
                plt.xlabel("Generation")
                plt.ylabel("Energy")
                plt.title("Network Energy over Successive Generations", fontsize=15)
                plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

                # quit pygame
                pygame.quit()

        cells = H_Net.state.reshape(img_size, img_size).T

        # fills surface with color
        surface.fill((211, 211, 211))

        # loop through network state array and update colors for each cell
        for r, c in np.ndindex(cells.shape):  # iterates through all cells in cells matrix
            if cells[r, c] == -1:
                col = (135, 206, 250)

            elif cells[r, c] == 1:
                col = (0, 0, 128)

            else:
                col = (255, 140, 0)
            pygame.draw.rect(
                surface, col, (r * cellsize, c * cellsize, cellsize, cellsize)
            )  # draw new cell_

        # update network state
        H_Net.update_network_state(100)
        H_Net.compute_energy()
        pygame.display.update()  # updates display from new .draw in update function
except Exception as inst:
    print(inst.args)


In [ ]:
from scipy.spatial import distance


In [ ]:
similarities = { i : (2 * distance.hamming(e, H_Net.state) - 1) ** 2 for i, e in enumerate(images)}
idx = max(similarities, key=similarities.get)
idx, similarities[idx]